In [ ]:
from google.colab import drive   #
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 파일 경로를 각자 컴퓨터 환경에 맞게 수정하세요!
file_path = '/content/drive/MyDrive/부리부리데마왕/dataset/'

In [ ]:
import numpy as np   # 결과해석
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]
import plotly.graph_objs as go
import plotly
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, StandardScaler

In [ ]:
pca_data = pd.read_csv(file_path + 'pca_train_test.csv')
pca_data.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
0,-0.186803,-1.000143,0.745271,0.396115,-1.285372,-0.708153,0.114003,0.325531,0.592047,0.439203,...,0.884376,-0.557766,0.036501,0.809532,0.478148,0.135564,1.472248,0.428958,-0.381381,0
1,-0.705777,-0.671973,0.878802,0.384820,0.332147,0.265318,0.581809,-0.587116,0.875597,-0.961119,...,-0.536303,-0.159865,0.910140,0.123068,-0.891751,0.650127,0.867556,-0.841684,0.475563,0
2,3.973764,0.752199,-0.057570,0.722408,1.857562,-1.213780,0.526862,-1.335082,0.488178,-0.693094,...,0.431250,0.468316,0.554997,0.719100,-0.347534,0.322468,-0.220685,0.436515,-0.433111,0
3,1.025226,0.970736,0.634796,-0.730365,-0.767119,0.839303,0.244287,0.100426,0.462459,0.848337,...,-0.503240,0.372951,0.879522,0.369692,-0.584512,1.226589,-0.281516,-0.572916,0.434625,0
4,-0.892340,-0.479857,0.500884,-1.393628,0.103438,0.917956,-1.092323,0.107218,0.561216,0.387365,...,-0.379418,0.749823,0.204488,0.400505,0.618971,0.204516,-0.777073,0.554618,-0.069441,0


In [ ]:
Normal = pca_data[pca_data['target']==0]

In [ ]:
Normal

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
0,-0.186803,-1.000143,0.745271,0.396115,-1.285372,-0.708153,0.114003,0.325531,0.592047,0.439203,...,0.884376,-0.557766,0.036501,0.809532,0.478148,0.135564,1.472248,0.428958,-0.381381,0
1,-0.705777,-0.671973,0.878802,0.384820,0.332147,0.265318,0.581809,-0.587116,0.875597,-0.961119,...,-0.536303,-0.159865,0.910140,0.123068,-0.891751,0.650127,0.867556,-0.841684,0.475563,0
2,3.973764,0.752199,-0.057570,0.722408,1.857562,-1.213780,0.526862,-1.335082,0.488178,-0.693094,...,0.431250,0.468316,0.554997,0.719100,-0.347534,0.322468,-0.220685,0.436515,-0.433111,0
3,1.025226,0.970736,0.634796,-0.730365,-0.767119,0.839303,0.244287,0.100426,0.462459,0.848337,...,-0.503240,0.372951,0.879522,0.369692,-0.584512,1.226589,-0.281516,-0.572916,0.434625,0
4,-0.892340,-0.479857,0.500884,-1.393628,0.103438,0.917956,-1.092323,0.107218,0.561216,0.387365,...,-0.379418,0.749823,0.204488,0.400505,0.618971,0.204516,-0.777073,0.554618,-0.069441,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284801,0.403169,0.402539,0.680315,-1.007532,2.098599,-1.277573,-0.646375,0.263997,0.781231,-0.454061,...,-0.410567,0.437024,0.627868,0.884708,0.772984,0.648746,-1.144199,0.589628,-0.273099,0
284802,-0.645635,-0.645760,0.370055,0.821711,-2.313645,-0.673857,0.365090,-1.252529,-0.886413,0.571019,...,-0.818363,0.809678,-2.978813,-3.835946,0.356766,-0.600699,-0.224977,0.767544,-0.854795,0
284803,-0.422941,0.331869,0.884928,0.401079,0.942990,-0.526874,-3.605100,0.052291,0.472184,0.005477,...,-0.103788,-0.729232,1.035139,0.383185,0.082624,-0.556574,-0.533770,0.598696,0.625018,0
284804,3.444423,-1.391055,-0.505879,0.415283,0.514673,-1.376723,-1.068739,0.129165,0.814837,0.994285,...,-0.546426,-0.742508,0.887634,-0.692838,0.431044,-3.674984,0.213256,0.937968,-0.285184,0


In [ ]:
# 중복된 row 이는지 확인
print(f"Total duplicate rows = {pca_data.duplicated().sum()}")

# 중복 제거 후의 데이터셋 크기 확인
print(f"Data shape after removing duplicates = {pca_data.shape}")

Total duplicate rows = 0
Data shape after removing duplicates = (284806, 29)


In [ ]:
pca_data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,target
0,-0.186803,-1.000143,0.745271,0.396115,-1.285372,-0.708153,0.114003,0.325531,0.592047,0.439203,...,0.884376,-0.557766,0.036501,0.809532,0.478148,0.135564,1.472248,0.428958,-0.381381,0
1,-0.705777,-0.671973,0.878802,0.384820,0.332147,0.265318,0.581809,-0.587116,0.875597,-0.961119,...,-0.536303,-0.159865,0.910140,0.123068,-0.891751,0.650127,0.867556,-0.841684,0.475563,0
2,3.973764,0.752199,-0.057570,0.722408,1.857562,-1.213780,0.526862,-1.335082,0.488178,-0.693094,...,0.431250,0.468316,0.554997,0.719100,-0.347534,0.322468,-0.220685,0.436515,-0.433111,0
3,1.025226,0.970736,0.634796,-0.730365,-0.767119,0.839303,0.244287,0.100426,0.462459,0.848337,...,-0.503240,0.372951,0.879522,0.369692,-0.584512,1.226589,-0.281516,-0.572916,0.434625,0
4,-0.892340,-0.479857,0.500884,-1.393628,0.103438,0.917956,-1.092323,0.107218,0.561216,0.387365,...,-0.379418,0.749823,0.204488,0.400505,0.618971,0.204516,-0.777073,0.554618,-0.069441,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284801,0.403169,0.402539,0.680315,-1.007532,2.098599,-1.277573,-0.646375,0.263997,0.781231,-0.454061,...,-0.410567,0.437024,0.627868,0.884708,0.772984,0.648746,-1.144199,0.589628,-0.273099,0
284802,-0.645635,-0.645760,0.370055,0.821711,-2.313645,-0.673857,0.365090,-1.252529,-0.886413,0.571019,...,-0.818363,0.809678,-2.978813,-3.835946,0.356766,-0.600699,-0.224977,0.767544,-0.854795,0
284803,-0.422941,0.331869,0.884928,0.401079,0.942990,-0.526874,-3.605100,0.052291,0.472184,0.005477,...,-0.103788,-0.729232,1.035139,0.383185,0.082624,-0.556574,-0.533770,0.598696,0.625018,0
284804,3.444423,-1.391055,-0.505879,0.415283,0.514673,-1.376723,-1.068739,0.129165,0.814837,0.994285,...,-0.546426,-0.742508,0.887634,-0.692838,0.431044,-3.674984,0.213256,0.937968,-0.285184,0


SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
# SMOTE 객체 생성
smote = SMOTE(random_state=42)

#특징과 클래스 분리
X = pca_data.drop('target', axis = 1)
y = pca_data['target']

# 데이터 업샘플링 수행
X_resampled, y_resampled = smote.fit_resample(X, y)

# 업샘플링된 데이터의 클래스 분포 확인
resampled_class_distribution = pd.Series(y_resampled).value_counts()

resampled_class_distribution

data_SMOTE = pd.concat([X_resampled, y_resampled], axis=1)
data_SMOTE.shape

(568628, 29)

case 1 : SMOTE+scale 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.model_selection import train_test_split

data_SMOTE_case1=data_SMOTE.copy()
#꼬리가 오른쪽으로 긴 변수
pro_skew = ['V3', 'V5', 'V7','V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24','V27', 'V28']
neg_skew = ['V2', 'V11', 'V19', 'V20', 'V21']
normal_col = ['V1', 'V4','V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26' ]

# pro_skew 로그변환 적용
# 변수 내의 최소값이 0 이하인 경우, 모든 값을 조금 올려서 로그 변환이 가능하도록 조정
if (data_SMOTE_case1[pro_skew].min() <= 0).any():  # .any()를 추가하여 Series 전체에서 하나라도 조건을 만족하는지 확인
    for col in pro_skew:
        if data_SMOTE_case1[col].min() <= 0:
            data_SMOTE_case1[col] += abs(data_SMOTE_case1[col].min()) + 0.1

for col in pro_skew:
    data_SMOTE_case1[col + '_log_transformed'] = np.log(data_SMOTE_case1[col])


# neg_skew는 음의 왜도.
# 제곱 변환 적용
for col in neg_skew:
  data_SMOTE_case1[col + '_squared_transformed'] = data_SMOTE_case1[col] ** 2

#normal_col
# 정규화할 변수 리스트
scaler = MinMaxScaler()
for col in normal_col:
    data_SMOTE_case1[col + '_minmax_transformed'] = scaler.fit_transform(data_SMOTE_case1[col].values.reshape(-1, 1)) # 2차원 배열로 바꿔서 실행

# 적용 다한 column은 drop

data_SMOTE_case1.drop(columns = pro_skew+neg_skew+normal_col, inplace = True)

print(data_SMOTE_case1.head())

X_case1 = data_SMOTE_case1.drop('target',axis=1)
Y_case1 = data_SMOTE_case1['target']

# 데이터 나누기
X_case1_train, X_case1_test, Y_case1_train, Y_case1_test = train_test_split(X_case1, Y_case1, test_size=0.3, random_state=25)



   target  V3_log_transformed  V5_log_transformed  V7_log_transformed  \
0       0            3.497852            3.317053            3.211946   
1       0            3.501885            3.374048            3.230613   
2       0            3.473256            3.424976            3.228439   
3       0            3.494503            3.335671            3.217180   
4       0            3.490429            3.366184            3.162138   

   V9_log_transformed  V10_log_transformed  V12_log_transformed  \
0            3.005139             3.224547             2.915893   
1            3.019086             3.167239             2.803769   
2            2.999981             3.178465             2.845195   
3            2.998699             3.240689             2.804290   
4            3.003610             3.222483             2.866141   

   V14_log_transformed  V15_log_transformed  V16_log_transformed  ...  \
0             2.474494             3.507461             2.918635  ...   
1           

Case 2 : SMOTE + scale2

In [ ]:
def quantileTransform(data, columns):
    qt = QuantileTransformer()
    scaled_data = pd.DataFrame(qt.fit_transform(data[columns]), columns=columns)
    return scaled_data

def boxcox_transform(data, columns, lambda_value):
    if lambda_value == 0:
        return np.log(data[columns])
    else:
        return (data[columns] ** lambda_value - 1) / lambda_value

data_SMOTE_case2 = data_SMOTE.copy()

X_case2 = data_SMOTE_case2.drop('Class',axis=1)
Y_case2 = data_SMOTE_case2['Class']

left_skewed_cols = ['V3', 'V5', 'V7', 'V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24', 'V27', 'V28']
right_skewed_cols = ['V2', 'V11', 'V19', 'V20', 'V21']

for col in left_skewed_cols:
    # 음의 왜도를 가진 열의 최소값이 0보다 작거나 같은 경우 양수로 변환
    if X_case2[col].min() <= 0:
        X_case2[col] += abs(X_case2[col].min()) + 1

# train_scaler
smtrain_left_skewed = boxcox_transform(X_case2, left_skewed_cols, 0.01)
smtrain_right_skewed = quantileTransform(X_case2, right_skewed_cols)

train_transformed_sm = pd.concat([X_case2, smtrain_left_skewed.add_suffix('_box_Transform'), smtrain_right_skewed.add_suffix('_qt_Transform')], axis=1)

# 그 외 수치
# minmax? standard?
from sklearn.preprocessing import StandardScaler
s_columns = ['V1', 'V4', 'V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26']
standardScaler = StandardScaler()

for col in s_columns:
    train_transformed_sm[col + '_standard_Transform'] = standardScaler.fit_transform(train_transformed_sm[[col]])

#train_transformed --> 전처리가 진행된 데이터
train_transformed_sm.drop(columns =left_skewed_cols + right_skewed_cols + s_columns, inplace = True)

data_SMOTE_case2=train_transformed_sm
print(data_SMOTE_case2.head())


X_case2_train, X_case2_test, Y_case2_train, Y_case2_test = train_test_split(data_SMOTE_case2, Y_case2, test_size=0.3, random_state=25)

   V3_log_Transform  V5_log_Transform  V7_log_Transform  V9_log_Transform  \
0          3.527665          3.352671          3.251434          3.053483   
1          3.531580          3.407726          3.269385          3.066776   
2          3.503800          3.457007          3.267293          3.048569   
3          3.524415          3.370643          3.256466          3.047348   
4          3.520461          3.400123          3.203601          3.052027   

   V10_log_Transform  V12_log_Transform  V14_log_Transform  V15_log_Transform  \
0           3.263550           2.968633           2.555342           3.536994   
1           3.208496           2.862586           2.724432           3.582686   
2           3.219271           2.901692           2.811186           3.536359   
3           3.279079           2.863078           2.740991           3.536163   
4           3.261565           2.921498           2.809234           3.570114   

   V16_log_Transform  V17_log_Transform  ...  V21_

case 3: SMOTE + scale3

In [ ]:
#Scaling 진행 X
data_SMOTE_case3=data_SMOTE.copy()

X_case3 = data_SMOTE_case3.drop('target',axis=1).values
Y_case3 = data_SMOTE_case3['target'].values

X_case3_train, X_case3_test, Y_case3_train, Y_case3_test = train_test_split(X_case3, Y_case3, test_size = 0.30, random_state = 25)
# X_TS_train['Time'] = scale.fit_transform(X_TS_train.reshape(1,-1))
# X_TS_test['Time'] = scale.fit_transform(X_TS_test.reshape(1,-1))

print(X_case3_train.shape,Y_case3_train.shape)
print(X_case3_test.shape,Y_case3_test.shape)

(398039, 28) (398039,)
(170589, 28) (170589,)


In [ ]:
from sklearn.naive_bayes import GaussianNB  # CASE 1


model = GaussianNB()   # 가우시안 naive Bayes
model.fit(X_case1_train , Y_case1_train)

GaussianNB()

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case1_test
predicted = model.predict(X_case1_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.69      0.33      0.44     85465
           1       0.56      0.85      0.67     85124

    accuracy                           0.59    170589
   macro avg       0.62      0.59      0.56    170589
weighted avg       0.62      0.59      0.56    170589

Accuracy: 0.5887483952658144
F1 Score: 0.6740539410411875


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.7222653683015712
Specision 0.1793742427482004
Precision 0.5575239603108115
False Positive Rate: 0.6736207804364359
False Negative Rate :  0.2777346316984288
True Positive Rate: 0.8521686010995724


In [ ]:
from sklearn.naive_bayes import BernoulliNB  # 베르누이 NB CASE1


model = BernoulliNB(alpha=2.0)
model.fit(X_case1_train, Y_case1_train)

BernoulliNB(alpha=2.0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case1_test
predicted = model.predict(X_case1_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       1.00      0.00      0.00     85465
           1       0.50      1.00      0.67     85124

    accuracy                           0.50    170589
   macro avg       0.75      0.50      0.33    170589
weighted avg       0.75      0.50      0.33    170589

Accuracy: 0.49901224580717396
F1 Score: 0.6657828564277641


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.9999765054155018
Specision 0.0
Precision 0.4990063721151084
False Positive Rate: 0.9999765986076171
False Negative Rate :  2.349458449827315e-05
True Positive Rate: 1.0


이 데이터는 베르누이 나이브 베이즈 분류가 맞지 않는 것을 증명할 수 있는 부분이라고 생각함. pca 특성상 한쪽으로 치우친 가중치를 준 것이기 때문에 베르누이를 했을 때 극명한 결과를 나오는 걸 볼 수 있음.

최대한 일반화 성능을 높히기 위해 pca_data에서는 time과 amount를 빼서 진행해본 결과 정확도가 높게 나오진 않았다. 그 이유는 분류 모델 특성상 이런 결과를 나왔다고 할 수 있다.

In [ ]:
from sklearn.naive_bayes import GaussianNB  # CASE 2


model = GaussianNB()   # 가우시안 naive Bayes
model.fit(X_case2_train , Y_case2_train)

GaussianNB()

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case2_test
predicted = model.predict(X_case2_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.67      0.46      0.55     85465
           1       0.59      0.77      0.67     85124

    accuracy                           0.62    170589
   macro avg       0.63      0.62      0.61    170589
weighted avg       0.63      0.62      0.61    170589

Accuracy: 0.6160655141890743
F1 Score: 0.6666938081740044


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.6232801111385997
Specision 0.29958012061989464
Precision 0.5881196297260656
False Positive Rate: 0.5367577370853566
False Negative Rate :  0.3767198888614003
True Positive Rate: 0.7695009633005968


In [ ]:
from sklearn.naive_bayes import BernoulliNB  # 베르누이 NB CASE2


model = BernoulliNB(alpha=2.0)
model.fit(X_case2_train, Y_case2_train)

BernoulliNB(alpha=2.0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case2_test
predicted = model.predict(X_case2_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       1.00      0.00      0.00     85465
           1       0.50      1.00      0.67     85124

    accuracy                           0.50    170589
   macro avg       0.75      0.50      0.33    170589
weighted avg       0.75      0.50      0.33    170589

Accuracy: 0.4995222435209773
F1 Score: 0.666009451381717


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.9989555584241842
Specision 0.0
Precision 0.4992609970674487
False Positive Rate: 0.9989586380389633
False Negative Rate :  0.0010444415758158966
True Positive Rate: 1.0


In [ ]:
from sklearn.naive_bayes import GaussianNB  # CASE 3


model = GaussianNB()   # 가우시안 naive Bayes
model.fit(X_case3_train , Y_case3_train)

GaussianNB()

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case3_test
predicted = model.predict(X_case3_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.66      0.50      0.57     85465
           1       0.60      0.74      0.66     85124

    accuracy                           0.62    170589
   macro avg       0.63      0.62      0.62    170589
weighted avg       0.63      0.62      0.62    170589

Accuracy: 0.6236861696826876
F1 Score: 0.6633664923936926


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.5944978100268812
Specision 0.340727470986837
Precision 0.5991209873736656
False Positive Rate: 0.4951968642134207
False Negative Rate :  0.40550218997311877
True Positive Rate: 0.743045439594004


In [ ]:
from sklearn.naive_bayes import BernoulliNB  # 베르누이 NB CASE3


model = BernoulliNB(alpha=2.0)
model.fit(X_case3_train, Y_case3_train)

BernoulliNB(alpha=2.0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

expected = Y_case3_test
predicted = model.predict(X_case3_test)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       0.56      0.56      0.56     85465
           1       0.56      0.56      0.56     85124

    accuracy                           0.56    170589
   macro avg       0.56      0.56      0.56    170589
weighted avg       0.56      0.56      0.56    170589

Accuracy: 0.5603761086588233
F1 Score: 0.5579714843128356


In [ ]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)

#TPR
tpr= tp / (tp + fn)


print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Positive Rate:', fpr)
print('False Negative Rate : ',tnr )
print('True Positive Rate:', tpr)

Sensitivity 0.49514613887900916
Specision 0.503913594239616
Precision 0.559908679039947
False Positive Rate: 0.4353127011057158
False Negative Rate :  0.5048538611209908
True Positive Rate: 0.5560476481368357
